# UnSupervisedIOHMM

In [9]:
from IOHMM import IOHMM_model
import numpy as np, pandas as pd
import torch

## Load data

In [10]:
data = pd.read_csv("data/Financial-Data/stocks/TSLA.csv")
data.head()
data = data.dropna()

input_data = data[['Open', 'High', 'Low', 'Close']][:-1]


# close price of the next day
output_data = data['Close'][1:]
# to start from 0 index
output_data.index = range(len(output_data))

output_data.name = 'Close-next-day'

data = pd.concat([input_data, output_data], axis=1)

data.head()
print(len(data))


input = torch.tensor(np.array(data[['Open', 'High', 'Low', 'Close']]), dtype=torch.float32)
output = torch.tensor(np.array(data['Close-next-day']), dtype=torch.float32)

print(output[0:5])
print(input[0:5])
torch.cat((torch.tensor([1.0]), output))

3259
tensor([1.5887, 1.4640, 1.2800, 1.0740, 1.0533])
tensor([[1.2667, 1.6667, 1.1693, 1.5927],
        [1.7193, 2.0280, 1.5533, 1.5887],
        [1.6667, 1.7280, 1.3513, 1.4640],
        [1.5333, 1.5400, 1.2473, 1.2800],
        [1.3333, 1.3333, 1.0553, 1.0740]])


tensor([  1.0000,   1.5887,   1.4640,  ..., 224.5700, 234.8600, 244.4000])

# Example 1

## Set up a simple model manully

In [11]:
IOHMM = IOHMM_model(num_states=2, inputs=input[:10], outputs=output[:10], max_iter=1000, tol=1e-6)
print(IOHMM.initial_pi)
print(IOHMM.transition_matrix)
print(IOHMM.emission_matrix)
print(IOHMM.sd)

Parameter containing:
tensor([0.5000, 0.5000], requires_grad=True)
Parameter containing:
tensor([[[ 0.9200, -0.4089,  0.2766, -0.4173,  0.0832],
         [-0.7292,  0.3889,  1.3879,  0.1671,  0.7959]],

        [[ 0.7234,  0.2948, -0.7810,  0.1341, -0.5918],
         [-0.8479, -0.7565, -0.5625,  1.0084, -0.2985]]], requires_grad=True)
Parameter containing:
tensor([[ 0.3396,  0.5620, -0.2205, -1.1630, -0.3678],
        [ 0.3874,  1.3382, -0.6726, -0.6105,  0.3642]], requires_grad=True)
Parameter containing:
tensor([1., 1.], requires_grad=True)


In [12]:
print(IOHMM._forward())
torch.sum(IOHMM._forward(), dim=1)

tensor([[0.0225, 0.9775],
        [0.0326, 0.9674],
        [0.1073, 0.8927],
        [0.2010, 0.7990],
        [0.2396, 0.7604],
        [0.1976, 0.8024],
        [0.1805, 0.8195],
        [0.1718, 0.8282],
        [0.1542, 0.8458],
        [0.1239, 0.8761]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [13]:
print(IOHMM._backward())
torch.sum(IOHMM._backward(), dim=1)

tensor([[0.0274, 0.9726],
        [0.0759, 0.9241],
        [0.1436, 0.8564],
        [0.2257, 0.7743],
        [0.2250, 0.7750],
        [0.1878, 0.8122],
        [0.1806, 0.8194],
        [0.1593, 0.8407],
        [0.1587, 0.8413],
        [0.0643, 0.9357]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [14]:
print(IOHMM._compute_gamma(IOHMM._forward(), IOHMM._backward()))
torch.sum(IOHMM._compute_gamma(IOHMM._forward(), IOHMM._backward()), dim=1)

tensor([[6.4776e-04, 9.9935e-01],
        [2.7628e-03, 9.9724e-01],
        [1.9752e-02, 9.8025e-01],
        [6.8333e-02, 9.3167e-01],
        [8.3808e-02, 9.1619e-01],
        [5.3868e-02, 9.4613e-01],
        [4.6290e-02, 9.5371e-01],
        [3.7839e-02, 9.6216e-01],
        [3.3239e-02, 9.6676e-01],
        [9.6266e-03, 9.9037e-01]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [15]:
print(IOHMM._compute_xi(IOHMM._forward(), IOHMM._backward()))
a = torch.sum(IOHMM._compute_xi(IOHMM._forward(), IOHMM._backward()), axis=1)
torch.sum(a, axis=1)

tensor([[[4.4176e-04, 5.0301e-02],
         [3.2481e-01, 6.2444e-01]],

        [[1.0065e-04, 1.8616e-01],
         [6.8013e-02, 7.4572e-01]],

        [[4.7274e-04, 3.5495e-01],
         [8.7420e-02, 5.5715e-01]],

        [[3.2421e-03, 4.2382e-01],
         [2.0757e-01, 3.6537e-01]],

        [[1.0271e-02, 3.5545e-01],
         [3.4537e-01, 2.8892e-01]],

        [[1.5412e-02, 2.6829e-01],
         [3.9898e-01, 3.1732e-01]],

        [[1.0909e-02, 2.7169e-01],
         [3.4557e-01, 3.7182e-01]],

        [[8.0762e-03, 2.5680e-01],
         [3.4225e-01, 3.9288e-01]],

        [[7.5151e-03, 2.6145e-01],
         [3.3077e-01, 4.0026e-01]],

        [[2.8180e-03, 1.1611e-01],
         [3.6154e-01, 5.1954e-01]]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [16]:
IOHMM._baum_welch()

## See the training results

In [17]:
print("Initial pi")
print(IOHMM.initial_pi) # to normalize
print("Transition matrix")
print(IOHMM.transition_matrix)
print("Emission matrix")
print(IOHMM.emission_matrix)
print("Sd")
print(IOHMM.sd)

Initial pi
Parameter containing:
tensor([6.6731e+03, 1.0089e+07], requires_grad=True)
Transition matrix
Parameter containing:
tensor([[[ 0.9200, -0.4089,  0.2766, -0.4173,  0.0832],
         [-0.7292,  0.3889,  1.3879,  0.1671,  0.7959]],

        [[ 0.7234,  0.2948, -0.7810,  0.1341, -0.5918],
         [-0.8479, -0.7565, -0.5625,  1.0084, -0.2985]]], requires_grad=True)
Emission matrix
Parameter containing:
tensor([[ 0.3396,  0.5620, -0.2205, -1.1630, -0.3678],
        [ 0.3874,  1.3382, -0.6726, -0.6105,  0.3642]], requires_grad=True)
Sd
Parameter containing:
tensor([1., 1.], requires_grad=True)
